In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [7]:
pseudo = pd.read_csv("../input/tps-aug-2021-lgbm-xgb-catboost/submission.csv")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

# Optuna

In [8]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [9]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[scale_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                  'bootstrap_type': 'Poisson',
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 9, 13),
           'learning_rate':trial.suggest_uniform('learning_rate', 6e-3, 8e-3),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 407, 487),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 57, 77),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 0.04, 0.24),
            'subsample': trial.suggest_uniform('subsample', 0.468, 0.588)
                 }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ctb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[scale_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[scale_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model =  ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    ctb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, ctb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 15)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-21 10:34:45,257] A new study created in memory with name: no-name-9d35a29d-812a-4f9a-8700-409ce8e81cf1


0:	learn: 6.3396414	test: 7.8583929	best: 7.8583929 (0)	total: 168ms	remaining: 27m 58s
1000:	learn: 5.7255046	test: 7.7896302	best: 7.7896302 (1000)	total: 2m 34s	remaining: 23m 10s
2000:	learn: 5.2774944	test: 7.7780582	best: 7.7780470 (1999)	total: 5m 6s	remaining: 20m 26s
3000:	learn: 4.8852618	test: 7.7725835	best: 7.7725717 (2993)	total: 7m 37s	remaining: 17m 48s
4000:	learn: 4.5160588	test: 7.7700361	best: 7.7699652 (3974)	total: 10m 10s	remaining: 15m 14s
5000:	learn: 4.1663284	test: 7.7683423	best: 7.7682287 (4982)	total: 12m 43s	remaining: 12m 43s
bestTest = 7.767339299
bestIteration = 5474
Shrink model to first 5475 iterations.
0:	learn: 6.3396084	test: 7.8584233	best: 7.8584233 (0)	total: 156ms	remaining: 26m 3s
1000:	learn: 5.7230081	test: 7.7901554	best: 7.7901554 (1000)	total: 2m 39s	remaining: 23m 54s
2000:	learn: 5.2756285	test: 7.7787080	best: 7.7787080 (2000)	total: 5m 16s	remaining: 21m 4s
3000:	learn: 4.8867034	test: 7.7737228	best: 7.7737228 (3000)	total: 7m 52s	r

[I 2021-08-21 11:06:23,780] Trial 0 finished with value: 7.7646494405604765 and parameters: {'max_depth': 13, 'learning_rate': 0.0077858696054715045, 'max_bin': 486, 'min_data_in_leaf': 72, 'reg_lambda': 0.051424000171051004, 'subsample': 0.5559560875184321}. Best is trial 0 with value: 7.7646494405604765.


0:	learn: 6.3400660	test: 7.8584220	best: 7.8584220 (0)	total: 34ms	remaining: 5m 39s
1000:	learn: 6.1367464	test: 7.7921680	best: 7.7921680 (1000)	total: 33.6s	remaining: 5m 1s
2000:	learn: 6.0120299	test: 7.7793358	best: 7.7793358 (2000)	total: 1m 6s	remaining: 4m 26s
3000:	learn: 5.9009533	test: 7.7730913	best: 7.7730913 (3000)	total: 1m 39s	remaining: 3m 51s
4000:	learn: 5.7967192	test: 7.7688533	best: 7.7688533 (4000)	total: 2m 11s	remaining: 3m 17s
5000:	learn: 5.6945533	test: 7.7659804	best: 7.7659804 (5000)	total: 2m 44s	remaining: 2m 44s
6000:	learn: 5.5961965	test: 7.7644694	best: 7.7644694 (6000)	total: 3m 17s	remaining: 2m 11s
bestTest = 7.762944292
bestIteration = 6783
Shrink model to first 6784 iterations.
0:	learn: 6.3400522	test: 7.8584465	best: 7.8584465 (0)	total: 36.2ms	remaining: 6m 1s
1000:	learn: 6.1380280	test: 7.7915798	best: 7.7915798 (1000)	total: 33.8s	remaining: 5m 3s
2000:	learn: 6.0135056	test: 7.7781954	best: 7.7781954 (2000)	total: 1m 7s	remaining: 4m 27

[I 2021-08-21 11:16:01,632] Trial 1 finished with value: 7.758298954986045 and parameters: {'max_depth': 10, 'learning_rate': 0.007831840041450305, 'max_bin': 468, 'min_data_in_leaf': 60, 'reg_lambda': 0.15526777559231913, 'subsample': 0.5336435347914744}. Best is trial 1 with value: 7.758298954986045.


...

0:	learn: 6.3398657	test: 7.8584116	best: 7.8584116 (0)	total: 83.2ms	remaining: 13m 51s
1000:	learn: 5.9452568	test: 7.7912963	best: 7.7912963 (1000)	total: 1m 20s	remaining: 12m 1s
2000:	learn: 5.6628910	test: 7.7781134	best: 7.7781134 (2000)	total: 2m 38s	remaining: 10m 33s
3000:	learn: 5.4070275	test: 7.7721197	best: 7.7721149 (2994)	total: 3m 56s	remaining: 9m 11s
4000:	learn: 5.1661924	test: 7.7686360	best: 7.7685863 (3966)	total: 5m 14s	remaining: 7m 52s
5000:	learn: 4.9322234	test: 7.7667631	best: 7.7666697 (4954)	total: 6m 33s	remaining: 6m 33s
6000:	learn: 4.7074791	test: 7.7641425	best: 7.7641425 (6000)	total: 7m 52s	remaining: 5m 14s
bestTest = 7.763239447
bestIteration = 6484
Shrink model to first 6485 iterations.
0:	learn: 6.3398746	test: 7.8584414	best: 7.8584414 (0)	total: 85ms	remaining: 14m 9s
1000:	learn: 5.9420819	test: 7.7922108	best: 7.7922108 (1000)	total: 1m 20s	remaining: 12m 6s
2000:	learn: 5.6578148	test: 7.7792641	best: 7.7792641 (2000)	total: 2m 39s	remaini

[I 2021-08-21 14:20:20,400] Trial 13 finished with value: 7.761581302818221 and parameters: {'max_depth': 12, 'learning_rate': 0.007447018051033308, 'max_bin': 411, 'min_data_in_leaf': 73, 'reg_lambda': 0.10337842419960527, 'subsample': 0.4974502845004783}. Best is trial 11 with value: 7.756000258000434.


0:	learn: 6.3399381	test: 7.8584623	best: 7.8584623 (0)	total: 90.9ms	remaining: 15m 8s
1000:	learn: 5.9708956	test: 7.7940546	best: 7.7940546 (1000)	total: 1m 27s	remaining: 13m 4s
2000:	learn: 5.7071802	test: 7.7819180	best: 7.7819180 (2000)	total: 2m 52s	remaining: 11m 27s
3000:	learn: 5.4729540	test: 7.7762937	best: 7.7762937 (2997)	total: 4m 16s	remaining: 9m 58s
4000:	learn: 5.2534703	test: 7.7724751	best: 7.7724751 (4000)	total: 5m 41s	remaining: 8m 31s
5000:	learn: 5.0386336	test: 7.7698183	best: 7.7697319 (4947)	total: 7m 6s	remaining: 7m 6s
6000:	learn: 4.8312397	test: 7.7675431	best: 7.7675406 (5842)	total: 8m 31s	remaining: 5m 40s
7000:	learn: 4.6293954	test: 7.7658761	best: 7.7658135 (6982)	total: 9m 57s	remaining: 4m 15s
bestTest = 7.765653899
bestIteration = 7087
Shrink model to first 7088 iterations.
0:	learn: 6.3399280	test: 7.8584658	best: 7.8584658 (0)	total: 93.9ms	remaining: 15m 38s
1000:	learn: 5.9713880	test: 7.7924380	best: 7.7924380 (1000)	total: 1m 28s	remaini

[I 2021-08-21 14:44:15,271] Trial 14 finished with value: 7.760211716920735 and parameters: {'max_depth': 12, 'learning_rate': 0.0068278579473649405, 'max_bin': 476, 'min_data_in_leaf': 67, 'reg_lambda': 0.08568310524141731, 'subsample': 0.49304030726218606}. Best is trial 11 with value: 7.756000258000434.


Number of finished trials: 15
Best trial: {'max_depth': 11, 'learning_rate': 0.0073229907325453696, 'max_bin': 408, 'min_data_in_leaf': 64, 'reg_lambda': 0.09207259245356939, 'subsample': 0.49677374687682424}


In [11]:
study.best_params

{'max_depth': 11,
 'learning_rate': 0.0073229907325453696,
 'max_bin': 408,
 'min_data_in_leaf': 64,
 'reg_lambda': 0.09207259245356939,
 'subsample': 0.49677374687682424}

# Log

7.768172118850735 row-wise noise ver4

7.776578973194598 random noise ver5

7.757181558352988 no noise ver7

7.756073089003697 no noise ver8 (narrow space)
